---

# Chapter 20: Domain-Driven Design (DDD)

## Opening Context

As software systems grow in complexity, the gap between the business domain (how experts think about the problem) and the technical implementation (how developers model it) often widens. This disconnect leads to software that is difficult to maintain, hard to evolve, and misaligned with business needs. **Domain‑Driven Design (DDD)**, popularized by Eric Evans, provides a set of patterns and principles to bridge that gap. It places the **domain**—the core business logic—at the center of the design, creating a shared language and model that both domain experts and developers can understand.

This chapter explores three foundational building blocks of DDD:

1. **Entities vs. Value Objects** – How to distinguish objects that have identity from those that are defined solely by their attributes.
2. **Aggregates and Repositories** – How to group related objects into consistency boundaries and provide persistence abstractions.
3. **Bounded Contexts** – How to divide large domains into smaller, cohesive models with explicit boundaries.

These patterns help you create a rich, expressive domain model that can evolve with the business while maintaining technical integrity.

---

## 20.1 Entities vs. Value Objects

### Intent
*Distinguish between objects that have a continuous identity (Entities) and objects that are defined by their attributes (Value Objects), and model them accordingly.*

### The Problem

In object‑oriented design, we often create classes that represent real‑world concepts. However, not all objects are equal. Some have a lifecycle and identity that must be tracked (e.g., a customer, an order), while others are merely descriptive and interchangeable (e.g., a monetary amount, an address). Treating them the same way leads to confusion, unnecessary complexity, and subtle bugs.

For example, consider a `Person` class with attributes like name, birth date, and address. If two people have the same name and address, are they the same person? No, because each person has a unique identity. On the other hand, if two `Money` objects both represent $10 USD, they are considered equal regardless of any “identity” – they are interchangeable.

### The Solution: Entities and Value Objects

DDD defines two fundamental types of objects:

- **Entities** – Objects that have a distinct identity that runs through time and different states. Equality is based on the identity, not the attributes.
- **Value Objects** – Objects that are defined only by their attributes. They are immutable and interchangeable. Two value objects with the same attribute values are considered equal.

#### Entities

Entities are the backbone of the domain model. They have a lifecycle, can change over time, and are tracked by an identifier.

```typescript
// domain/entities/customer.ts
export class Customer {
  constructor(
    public readonly id: CustomerId, // unique identifier
    private _name: string,
    private _email: string,
    private _loyaltyPoints: number = 0
  ) {}

  // Business methods
  changeName(newName: string): void {
    if (newName.length < 2) throw new Error('Name too short');
    this._name = newName;
  }

  addLoyaltyPoints(points: number): void {
    this._loyaltyPoints += points;
  }

  // Equality based on identity
  equals(other: Customer): boolean {
    return this.id.equals(other.id);
  }
}

// Value object for CustomerId (itself a value object)
export class CustomerId {
  constructor(private readonly value: string) {
    if (!value) throw new Error('CustomerId cannot be empty');
  }

  equals(other: CustomerId): boolean {
    return this.value === other.value;
  }

  toString(): string {
    return this.value;
  }
}
```

**Explanation**:
- `Customer` has an identity (`CustomerId`). Even if two customers have the same name and email, they are different because their IDs differ.
- The `equals` method compares IDs, not attributes.
- `CustomerId` is a value object because it has no identity apart from its string value.

#### Value Objects

Value objects are immutable and have no identity. They are often used to represent descriptive aspects of the domain.

```typescript
// domain/value-objects/address.ts
export class Address {
  constructor(
    public readonly street: string,
    public readonly city: string,
    public readonly postalCode: string,
    public readonly country: string
  ) {
    // Validate in constructor to ensure valid state
    if (!street || !city || !postalCode || !country) {
      throw new Error('All address fields are required');
    }
  }

  // Value objects are immutable; to "change" an address, replace it
  withStreet(newStreet: string): Address {
    return new Address(newStreet, this.city, this.postalCode, this.country);
  }

  // Equality based on all attributes
  equals(other: Address): boolean {
    return this.street === other.street &&
           this.city === other.city &&
           this.postalCode === other.postalCode &&
           this.country === other.country;
  }
}

// Another example: Money
export class Money {
  constructor(
    public readonly amount: number,
    public readonly currency: string
  ) {
    if (amount < 0) throw new Error('Amount cannot be negative');
    if (currency.length !== 3) throw new Error('Invalid currency code');
  }

  add(other: Money): Money {
    if (this.currency !== other.currency) {
      throw new Error('Cannot add different currencies');
    }
    return new Money(this.amount + other.amount, this.currency);
  }

  equals(other: Money): boolean {
    return this.amount === other.amount && this.currency === other.currency;
  }
}
```

**Explanation**:
- `Address` and `Money` are immutable. Once created, their state cannot change.
- To "modify" a value object, you create a new instance (e.g., `withStreet`).
- Equality is based on all attributes; two addresses with the same values are considered equal.

### Benefits

- **Clarity** – Distinguishing entities from value objects makes the domain model more expressive.
- **Immutability** – Value objects are immutable, which eliminates side‑effects and makes them safe to share.
- **Persistence optimisation** – Value objects can be embedded in entities (e.g., an address stored as columns in a customer table) without separate identity.

### Pitfalls

- **Over‑modelling** – Not every concept needs to be a value object; use judgment.
- **Performance** – Creating many immutable objects can be expensive, but modern runtimes handle this well.

---

## 20.2 Aggregates and Repositories

### Intent
*Define consistency boundaries around groups of related objects (Aggregates) and provide a way to retrieve and persist them (Repositories).*

### The Problem

In a rich domain model, objects reference each other. Without boundaries, it’s easy to create a tangled web of relationships that is hard to maintain and reason about. Changes to one object might need to be consistent with changes to others, but enforcing consistency across many objects can be expensive and lead to contention.

### The Solution: Aggregates

An **Aggregate** is a cluster of domain objects that can be treated as a single unit. It has a root (the **Aggregate Root**) and a boundary. External objects can only hold references to the root, not to internal objects. All invariants (consistency rules) are enforced within the aggregate boundary.

#### Aggregate Rules

- The Aggregate Root is the only object that external clients can reference.
- Internal objects (entities, value objects) can reference each other freely, but not outside the aggregate.
- All changes to the aggregate must go through the root.
- For consistency, within a single transaction, only one aggregate should be modified. (This is a guideline; in practice, eventual consistency may be used across aggregates.)

#### Example: Order Aggregate

Consider an order with line items. The order is the aggregate root; line items are internal entities.

```typescript
// domain/aggregates/order.ts (Aggregate Root)
import { OrderId } from '../value-objects/order-id';
import { CustomerId } from '../value-objects/customer-id';
import { Money } from '../value-objects/money';
import { OrderItem } from '../entities/order-item'; // internal entity

export class Order {
  private _items: OrderItem[] = [];
  private _status: 'pending' | 'paid' | 'shipped' | 'cancelled' = 'pending';

  constructor(
    public readonly id: OrderId,
    public readonly customerId: CustomerId
  ) {}

  // Public methods to manipulate the aggregate
  addItem(productId: string, quantity: number, price: Money): void {
    if (this._status !== 'pending') {
      throw new Error('Cannot add items to a non‑pending order');
    }
    this._items.push(new OrderItem(productId, quantity, price));
  }

  removeItem(productId: string): void {
    if (this._status !== 'pending') {
      throw new Error('Cannot remove items from a non‑pending order');
    }
    this._items = this._items.filter(item => !item.productId.equals(productId));
  }

  get total(): Money {
    return this._items.reduce(
      (sum, item) => sum.add(item.subtotal),
      new Money(0, 'USD') // assuming same currency
    );
  }

  markAsPaid(): void {
    if (this._status !== 'pending') throw new Error('Order already processed');
    this._status = 'paid';
  }

  // Internal access to items (only for repository or internal use)
  get items(): ReadonlyArray<OrderItem> {
    return this._items;
  }

  // Domain events can be raised here
}
```

```typescript
// domain/entities/order-item.ts (internal entity, not exposed outside aggregate)
import { Money } from '../value-objects/money';

export class OrderItem {
  constructor(
    public readonly productId: string,
    public readonly quantity: number,
    public readonly unitPrice: Money
  ) {}

  get subtotal(): Money {
    return new Money(this.unitPrice.amount * this.quantity, this.unitPrice.currency);
  }

  // No identity; might be an entity if line items have identity (e.g., if they can be modified individually)
  // Here we treat it as an entity because quantity can change, but identity is within the aggregate.
}
```

**Explanation**:
- `Order` is the aggregate root. Clients interact with the order to add/remove items, mark as paid, etc.
- `OrderItem` is an internal entity (it has no identity outside the order). Its identity is local to the aggregate (e.g., productId + orderId).
- The `Order` enforces invariants: cannot add items after payment.
- External code cannot get a reference to an `OrderItem` directly; all operations go through the root.

### Repositories

Repositories provide a way to retrieve and persist aggregates. They act as an abstraction over the data layer, giving the illusion of an in‑memory collection of aggregates.

```typescript
// domain/repositories/order.repository.ts
import { Order } from '../aggregates/order';
import { OrderId } from '../value-objects/order-id';

export interface OrderRepository {
  findById(id: OrderId): Promise<Order | null>;
  save(order: Order): Promise<void>;
  delete(id: OrderId): Promise<void>;
}
```

**Infrastructure implementation (e.g., using TypeORM)**:

```typescript
// infrastructure/repositories/typeorm-order.repository.ts
import { OrderRepository } from '../../domain/repositories/order.repository';
import { Order } from '../../domain/aggregates/order';
import { OrderId } from '../../domain/value-objects/order-id';
import { getRepository } from 'typeorm';
import { OrderEntity } from '../entities/order.entity'; // ORM entity

export class TypeOrmOrderRepository implements OrderRepository {
  async findById(id: OrderId): Promise<Order | null> {
    const ormEntity = await getRepository(OrderEntity).findOne({
      where: { id: id.toString() },
      relations: ['items'] // load internal entities
    });
    if (!ormEntity) return null;
    return this.toDomain(ormEntity);
  }

  async save(order: Order): Promise<void> {
    const ormEntity = this.toPersistence(order);
    await getRepository(OrderEntity).save(ormEntity);
  }

  // ... delete

  private toDomain(orm: OrderEntity): Order {
    const order = new Order(new OrderId(orm.id), new CustomerId(orm.customerId));
    // populate items
    for (const itemOrm of orm.items) {
      order.addItem(itemOrm.productId, itemOrm.quantity, new Money(itemOrm.price, 'USD'));
    }
    // set status
    return order;
  }

  private toPersistence(order: Order): OrderEntity {
    // map domain order to ORM entity
  }
}
```

**Explanation**:
- The repository interface lives in the domain layer, defining operations in domain terms.
- The implementation is in the infrastructure layer, mapping between domain objects and persistence.
- This keeps the domain clean of persistence concerns.

### Aggregate Design Considerations

- **Size** – Aggregates should be as small as possible while maintaining invariants. Large aggregates cause contention.
- **References** – Reference other aggregates by identity only (e.g., `customerId`), not by object reference. This allows aggregates to be loaded independently.
- **Transactional boundaries** – Modify only one aggregate per transaction. For operations spanning multiple aggregates, use eventual consistency (e.g., sagas).

### Benefits

- **Consistency** – Invariants are enforced within the aggregate.
- **Performance** – Smaller aggregates reduce contention.
- **Clarity** – Explicit boundaries make the model easier to understand.

### Pitfalls

- **Over‑aggregation** – Making aggregates too large leads to performance issues.
- **Under‑aggregation** – Too many tiny aggregates can make it hard to enforce consistency.
- **Transaction management** – Requires careful design to maintain consistency across aggregates.

---

## 20.3 Bounded Contexts

### Intent
*Explicitly define the boundaries within which a particular domain model applies, allowing multiple models to coexist without conflict.*

### The Problem

Large organizations have multiple teams working on different parts of the system. The same term (e.g., “customer”) may mean different things in different contexts. For example, in the Sales context, a customer has a credit limit and sales history; in the Support context, the same customer has tickets and contact preferences. Trying to build a single, unified “Customer” model that satisfies both contexts leads to a bloated, inconsistent model that is hard to maintain and understand.

### The Solution: Bounded Contexts

A **Bounded Context** is a semantic boundary within which a particular domain model is defined and applicable. It aligns with team boundaries, subdomains, and language. Within a bounded context, the model is internally consistent, but terms may have different meanings in other contexts.

#### Strategic Patterns

- **Context Map** – A diagram or document that shows the relationships between bounded contexts.
- **Integration patterns** – How contexts communicate:
  - **Shared Kernel** – Share a subset of the model (rare).
  - **Customer/Supplier** – One context provides data to another.
  - **Conformist** – One context conforms to another’s model.
  - **Anti‑Corruption Layer (ACL)** – A translation layer that protects a context from the “corruption” of another’s model.
  - **Open Host Service** – A protocol that gives access to the context as a service.
  - **Published Language** – A shared, documented language (e.g., XML schema, JSON schema) for integration.

#### Example: E‑commerce System with Multiple Bounded Contexts

Imagine an e‑commerce platform with at least three bounded contexts:

1. **Sales Context** – Handles orders, pricing, promotions.
2. **Inventory Context** – Manages stock levels, warehouses.
3. **Shipping Context** – Handles shipments, tracking.

Each context has its own model of `Product`. In Sales, a product has a price and description. In Inventory, a product has a stock level and location. In Shipping, a product has dimensions and weight.

**Sales Context**:

```typescript
// sales/domain/product.ts
export class Product {
  constructor(
    public readonly id: ProductId,
    public readonly name: string,
    public readonly price: Money
  ) {}
}
```

**Inventory Context**:

```typescript
// inventory/domain/product.ts
export class Product {
  constructor(
    public readonly sku: Sku, // different identifier
    public readonly quantityOnHand: number,
    public readonly reorderThreshold: number
  ) {}
}
```

**Shipping Context**:

```typescript
// shipping/domain/product.ts
export class Product {
  constructor(
    public readonly id: ProductId, // same id as sales? maybe not
    public readonly weight: number,
    public readonly dimensions: Dimensions
  ) {}
}
```

**Integration**:
When an order is placed in the Sales context, it needs to reserve stock in Inventory. This is done via an **Anti‑Corruption Layer** in the Inventory context that translates the Sales event into Inventory commands.

```typescript
// inventory/anti-corruption/sales-order.listener.ts
import { SalesOrderPlacedEvent } from 'sales/events'; // but we shouldn't depend on Sales directly!
// Instead, use a shared message format (Published Language).

export class SalesOrderPlacedHandler {
  constructor(private inventoryService: InventoryService) {}

  async handle(message: SalesOrderPlacedMessage): Promise<void> {
    // Translate message into Inventory domain commands
    for (const item of message.items) {
      await this.inventoryService.reserveStock(item.productId, item.quantity);
    }
  }
}
```

**Explanation**:
- The Inventory context does not depend on the Sales domain model. It receives a message in a neutral format (e.g., JSON) and translates it into its own domain commands.
- This protects Inventory from changes in the Sales model.

### Benefits

- **Team autonomy** – Teams can develop their own models independently.
- **Model clarity** – Each model is consistent and focused on its subdomain.
- **Technology independence** – Different contexts can use different technologies, databases, and languages.
- **Scalability** – Contexts can be deployed independently.

### Pitfalls

- **Integration complexity** – Communication between contexts requires careful design (ACLs, messages).
- **Data duplication** – Data is often duplicated across contexts (e.g., product name). This is acceptable if eventual consistency is managed.
- **Discoverability** – It can be hard for developers to understand the overall system without a context map.

### Identifying Bounded Contexts

- Look for **ubiquitous language** – terms that are consistently used by domain experts in a specific part of the business.
- Align with **organizational structure** (Conway’s Law).
- Consider **subdomains** – core, supporting, and generic.

---

## Chapter Summary

Domain‑Driven Design provides a set of patterns for building software that reflects complex business domains:

1. **Entities vs. Value Objects** – Entities have identity and a lifecycle; value objects are immutable and defined by their attributes. Distinguishing them clarifies the model and promotes immutability.

2. **Aggregates and Repositories** – Aggregates define consistency boundaries with a root that controls access. Repositories provide a collection‑like interface for retrieving and persisting aggregates, isolating the domain from persistence concerns.

3. **Bounded Contexts** – Explicit boundaries where a particular model applies. Contexts integrate via translation layers (e.g., Anti‑Corruption Layer), allowing multiple models to coexist without conflict.

**Key Insight**: DDD is not about technology; it’s about creating a shared understanding of the domain and encoding that understanding into a software model. The tactical patterns (entities, value objects, aggregates) give you the building blocks, while the strategic patterns (bounded contexts) help you scale across large organizations.

---

## Next Chapter Preview

**Chapter 21: Common Anti-Patterns (Spaghetti Code, God Object, Golden Hammer, Premature Optimization)**

Design patterns are powerful, but knowing when *not* to use them is equally important. Chapter 21 explores common anti‑patterns—bad practices that seem like good ideas at first but lead to unmaintainable code. You’ll learn to recognise and avoid them, and you’ll be better equipped to write clean, sustainable software.



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='19. security_design_patterns.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../7. anti_patterns_refactoring_best_practices/21. common_anti_patterns.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
